In [0]:
%pip install databricks_langchain

In [0]:
from databricks_langchain.uc_ai import (
    DatabricksFunctionClient,
    UCFunctionToolkit,
    set_uc_function_client,
)
import pandas as pd
from databricks.connect import DatabricksSession

# Initialize the default notebook Spark session
spark = DatabricksSession.builder.getOrCreate()

# Create a Databricks function client
client = DatabricksFunctionClient()

# Set the function client for UC
set_uc_function_client(client)

# Define tools using UCFunctionToolkit with specified function names
tools = UCFunctionToolkit(
    function_names=[
        "alexander_genser.train_agent.get_connections", 
        "alexander_genser.train_agent.get_station_board"
    ]
).tools

In [0]:
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.chat_models import ChatDatabricks


# Utilize a Foundational Model API via ChatDatabricks 
llm = ChatDatabricks(endpoint="databricks-meta-llama-3-1-70b-instruct")

# read system prompt from file
with open("system_prompt.txt", "r") as f:
    system_prompt = f.read()

# Define the prompt for the model, note the description to use the tools
prompt = ChatPromptTemplate.from_messages(
    [   ("system", system_prompt),
        ("placeholder", "{chat_history}"),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)

In [0]:
from langchain.agents import AgentExecutor, create_tool_calling_agent

agent = create_tool_calling_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=False)

In [0]:
agent_executor.invoke({"input": "When is the next train leaving from Zurich HB to Geneva?"})